# Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) data structure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on a trie;
3. Measure suggestion speed;

![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [4]:
!pip install pygtrie

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1. Build a trie upon a dataset

### 1.1. [5] Read the dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset. Pass the assert.

In [5]:
!gunzip user-ct-test-collection-01.txt.gz

In [6]:
import pandas as pd


aol_data = pd.read_csv('user-ct-test-collection-01.txt', on_bad_lines='skip',sep='\t')

#TODO: Read the dataset, e.g. as pandas dataframe

assert aol_data.shape[0] == 3558411, "Dataset size does not match"

In [7]:
aol_data.Query.fillna('',inplace=True)

### 1.2. [10] Build a Trie

We want a suggest function to be **non-sensitive to stop words** because we don't want to upset the users if they confuses/omits prepositions. Consider *"public events in Innopolis"* vs *"public events at Innopolis"* or *"public events Innopolis"* - they all mean the same.

Build a Trie based on the dataset, **storing query statistics such as query _frequency_, urls and ranks in the nodes**. Some queries may have no associated urls, others may have multiple ranked urls. Think of the way to store this information.

Pass the asserts.

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
from nltk.tokenize import word_tokenize

In [10]:
def preprocess(text):   
  temp = [token.lower().strip() for token in word_tokenize(text) if token.lower() not in stop_words]
  return ' '.join(temp).lower()

In [11]:
aol_data.head()

,AnonID,Query,QueryTime,ItemRank,ClickURL
0,142,rentdirect.com,2006-03-01 07:17:12,NaN,NaN
1,142,www.prescriptionfortime.com,2006-03-12 12:31:06,NaN,NaN
2,142,staple.com,2006-03-17 21:19:29,NaN,NaN
3,142,staple.com,2006-03-17 21:19:45,NaN,NaN
4,142,www.newyorklawyersite.com,2006-03-18 08:02:58,NaN,NaN


In [12]:
import pygtrie
aol_trie = pygtrie.CharTrie()

for i, row in aol_data.iterrows():
  if bool(row["Query"]):
    filtered_sentence = preprocess(row["Query"])
    if filtered_sentence in aol_trie:
      prev_value = aol_trie[filtered_sentence]
      aol_trie[filtered_sentence] = {
          'queries': prev_value['queries']+ [row["Query"]],
          'ranks': prev_value['ranks']+[row['ItemRank']],
          'urls': prev_value['urls']+[row["ClickURL"]]
      }    
    else:
      aol_trie[filtered_sentence] = {
          'queries': [row["Query"]],
          'ranks': [row['ItemRank']],
          'urls': [row["ClickURL"]]
      }

#TODO: build a trie based on the dataset

In [13]:

# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a property of list type. But you can do something different. 
    bag += val['urls']
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

sample question surveys ~ {'queries': ['sample question surveys', 'sample question surveys', 'sample question surveys', 'sample question surveys', 'sample question surveys'], 'ranks': [7.0, 4.0, 10.0, 1.0, nan], 'urls': ['http://www.surveyconnect.com', 'http://www.custominsight.com', 'http://www.askemployees.com', 'http://www.lg-employers.gov.uk', nan]}
sample questions immigration interview ~ {'queries': ['sample questions for immigration interview'], 'ranks': [nan], 'urls': [nan]}
sample questions interview ~ {'queries': ['sample questions for interview'], 'ranks': [1.0], 'urls': ['http://www.quintcareers.com']}
sample questions family interview ~ {'queries': ['sample questions for family interview', 'sample questions for family interview', 'sample questions for family interview'], 'ranks': [2.0, 5.0, 3.0], 'urls': ['http://www.grandparents-day.com', 'http://www.quintcareers.com', 'http://jobsearchtech.about.com']}
sample questions sociology race ethnicity ~ {'queries': ['sample ques

## 2. [15] Write a suggest function which is 

---

non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

Pass the asserts.

Question for analysis: What is the empirical threshold for minimal prefix for suggest?

In [14]:
stops = set('a on at of to is from for and with using the in &'.split())

In [15]:
import numpy as np
from collections import Counter,OrderedDict

def complete_user_query(query: str, trie, top_k=5) -> list[str]:
  query = [w.lower() for w in word_tokenize(query) if not w.lower() in stops]
  query = ' '.join(query).lower()
  result = {}
  if trie.has_node(query):
    for key, obj in trie.iteritems(query):
      for i,item in enumerate(obj['queries']):
        if item not in result:
          result[item] = {
              'url': obj['urls'][i],
              'ranks': obj['ranks'][i],
              'count':1
          }
        else:
          rank = obj['ranks'][i]
          result[item]['count'] += 1
          if rank < result[item]['ranks']:
            result[item]['url'] = obj['urls'][i]
  return sorted(result, key =lambda key: result[key]['count'], reverse=True)[:top_k]
  
        
inp = "trie"
print("Query:", inp)
print("Results:")
res = complete_user_query(inp, aol_trie)
print(res)

#NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

assert "boys and girls club of conyers georgia" \
            in complete_user_query("boys girls club conyers", aol_trie, 10), "Should be here"

Query: trie
Results:
['tried and true tattoo', 'triest', 'triethanalomine', 'tried and failed', 'when you tried and failed']


## 3. Measure suggest speed ##

### 3.1. [10] Full Trie test

Check how fast your search is working. Consider changing your code if it takes too long on average.

Sucess criterion:
- there is an average and a standard deviation for **multiple runs** of the given bucket.
- there is an average and a standard deviation for **multiple runs** of naive search in the unindexed dataset.

In [16]:
def naive_search(data, query, top_k):
  groups = data[data.Query.str.lower().str.contains("the best")].groupby('Query')
  result = {}
  for (key, group) in groups:
    min_value = group['ItemRank'].min()
    if(pd.isnull(min_value)):
      result[key] = {
          'count': group.shape[0],
          'url': group['ClickURL'].iloc[0]       
      }
    else:
      temp = group[group['ItemRank'] == min_value].iloc[0]
      result[key] = {
          'count': group.shape[0],
          'url': temp['ClickURL']       
      }
  return sorted(result, key =lambda key: result[key]['count'], reverse=True)[:top_k]

In [17]:
inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

for inp in inp_queries:
  print("Query : "+ inp)
  print("Trie Index structure")
  %timeit -o complete_user_query(inp, aol_trie)
  print("Naive search")
  %timeit -o naive_search(aol_data,inp, 5)  
  print('---'*10)

#TODO: measure average execution time and standard deviation (in milliseconds) per query and print it out
# Repeat this for index and for no index.

Query : inf
Trie Index structure
25.5 ms ± 928 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Naive search
3.24 s ± 853 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
------------------------------
Query : the best 
Trie Index structure
39.7 ms ± 599 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Naive search
2.92 s ± 443 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
------------------------------
Query : information retrieval
Trie Index structure
79.1 µs ± 469 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Naive search
2.69 s ± 342 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
------------------------------
Query : sherlock hol
Trie Index structure
162 µs ± 32.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Naive search
2.73 s ± 568 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
------------------------------
Query : carnegie mell
Trie Index structure
124 µs ± 29.1 µs per loop (mean ± std. dev. of 7 runs, 10000 l

## 4. [10] Add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

You can use use [pyspellchecker](https://pypi.org/project/pyspellchecker/) `candidates()` call, or use any other spellchecker implementation.

In [18]:
pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622380 sha256=c1f37cc43dbee7e89212e11060e32ae0a13c3d4a86fdc7a68e3c7e5b5dadbaee
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [19]:
count = Counter()

for sentence in aol_data['Query']:
  sentence_split = sentence.split()
  for i, item in enumerate(sentence_split[1:]):
    count[f'{sentence_split[i].lower()} {item.lower()}'] +=1
  for item in sentence_split:
    if item.lower() not in stop_words:
      count[item.lower()] +=1  

In [20]:
temp = Counter({k: c for k, c in count.items() if c > 4})

In [21]:
from autocorrect import Speller

In [22]:
spell = Speller()

In [23]:
spell.nlp_data.update(temp)

In [24]:
def complete_user_query_with_spellchecker(query, trie, top_k=5) -> list[str]:
  query = spell(query)
  return complete_user_query(query, trie, top_k)

In [25]:
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]

for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, aol_trie, 5) == \
            complete_user_query_with_spellchecker(q, aol_trie, 5), "Assert {} and {} give different results".format(q, qc)